## Data Processing for Adding the Urban Percentage to the PLACE Dataset

This part is used to join the PLACE data with level of Urbanization index:

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import time
import datetime
import math
import pandas as pd
import plotly.express as px
import re

### 1.1 Preprocess CDC Places Data:

In [3]:
cdc_places = pd.read_csv(r'/mnt/d/OneDrive - University College London/T1/CASA0007/0. Research Project/cleaned_data/combined_df.csv', encoding = "UTF-8")

In [4]:
cdc_places['GeoLocation'] = cdc_places['Location'].str.replace('_', ' | ')



[Credit goes to](https://github.com/ChuckConnell/articles/blob/master/fips2county.tsv)


In [5]:
state_county_lookup = pd.read_csv(r'/mnt/d/OneDrive - University College London/T1/CASA0007/0. Research Project/cleaned_data/fips2county.tsv',  sep='\t', header=0)
fips_lookup  = state_county_lookup[['STATE_COUNTY', 'CountyFIPS']].set_index('STATE_COUNTY').to_dict()['CountyFIPS']

In [6]:
state_county_lookup['CountyFIPS']

0        1001
1        1003
2        1005
3        1007
4        1009
        ...  
3138    56037
3139    56039
3140    56041
3141    56043
3142    56045
Name: CountyFIPS, Length: 3143, dtype: int64

In [7]:
from county_matcher import fips_match

True


In [9]:
for index, row in cdc_places.iterrows():
    for key in fips_lookup:
        if fips_match(key,row['GeoLocation']):
            matched_key = key
            break
    print(matched_key)
    print(row['GeoLocation'])
        # store the matched key in the dataframe
    cdc_places.loc[index, 'GeoLocation'] = matched_key
    cdc_places.loc[index, 'GEOID'] = fips_lookup[matched_key]
    

AK | ALEUTIANS EAST BOROUGH
AK | ALEUTIANS EAST BOROUGH
AK | ALEUTIANS WEST CENSUS AREA
AK | ALEUTIANS WEST CENSUS AREA
AK | ANCHORAGE MUNICIPALITY
AK | ANCHORAGE MUNICIPALITY
AK | BETHEL CENSUS AREA
AK | BETHEL CENSUS AREA
AK | BRISTOL BAY BOROUGH
AK | BRISTOL BAY BOROUGH
AK | CHUGACH CENSUS AREA
AK | CHUGACH CENSUS AREA
AK | COPPER RIVER CENSUS AREA
AK | COPPER RIVER CENSUS AREA
AK | DENALI BOROUGH
AK | DENALI BOROUGH
AK | DILLINGHAM CENSUS AREA
AK | DILLINGHAM CENSUS AREA
AK | FAIRBANKS NORTH STAR BOROUGH
AK | FAIRBANKS NORTH STAR BOROUGH
AK | HAINES BOROUGH
AK | HAINES BOROUGH
AK | HOONAH-ANGOON CENSUS AREA
AK | HOONAH-ANGOON CENSUS AREA
AK | JUNEAU CITY AND BOROUGH
AK | JUNEAU CITY AND BOROUGH
AK | KENAI PENINSULA BOROUGH
AK | KENAI PENINSULA BOROUGH
AK | KETCHIKAN GATEWAY BOROUGH
AK | KETCHIKAN GATEWAY BOROUGH
AK | KODIAK ISLAND BOROUGH
AK | KODIAK ISLAND BOROUGH
AK | KUSILVAK CENSUS AREA
AK | KUSILVAK CENSUS AREA
AK | LAKE AND PENINSULA BOROUGH
AK | LAKE AND PENINSULA BOROUGH
AK

In [10]:
cdc_places['GEOID'] = cdc_places['GEOID'].astype('int64')
# Convert FIPS codes in DataFrame to string and zero-pad to 5 characters
cdc_places['GEOID'] = cdc_places['GEOID'].apply(lambda x: str(x).zfill(5))

In [11]:
cdc_places.head()

,All teeth lost among adults aged >=65 years,Cervical cancer screening among adult women aged 21-65 years,"Fecal occult blood test, sigmoidoscopy, or colonoscopy among adults aged 50-75 years",Mammography use among women aged 50-74 years,"Older adult men aged >=65 years who are up to date on a core set of clinical preventive services: Flu shot past year, PPV shot ever, Colorectal cancer screening","Older adult women aged >=65 years who are up to date on a core set of clinical preventive services: Flu shot past year, PPV shot ever, Colorectal cancer screening, and Mammogram past 2 years",Sleeping less than 7 hours among adults aged >=18 years,Visits to dentist or dental clinic among adults aged >=18 years,Location,Any disability among adults aged >=18 years,...,No leisure-time physical activity among adults aged >=18 years,Obesity among adults aged >=18 years,Physical health not good for >=14 days among adults aged >=18 years,Self-care disability among adults aged >=18 years,Stroke among adults aged >=18 years,Taking medicine for high blood pressure control among adults aged >=18 years with high blood pressure,Vision disability among adults aged >=18 years,Visits to doctor for routine checkup within the past year among adults aged >=18 years,GeoLocation,GEOID
0,16.95,68.35,57.35,59.95,28.45,28.15,34.20,48.50,AK_Aleutians East,28.80,...,29.00,31.70,11.40,3.85,3.4,62.30,6.15,60.7,AK | ALEUTIANS EAST BOROUGH,02013
1,12.05,69.95,62.40,59.50,32.55,30.50,33.95,55.55,AK_Aleutians West,24.20,...,25.05,30.95,9.55,2.90,2.7,59.60,4.50,61.1,AK | ALEUTIANS WEST CENSUS AREA,02016
2,9.75,78.15,68.85,66.25,40.65,32.90,30.90,64.85,AK_Anchorage,25.25,...,20.40,32.95,10.30,2.90,2.6,60.45,4.05,62.8,AK | ANCHORAGE MUNICIPALITY,02020
3,28.05,72.45,56.65,51.25,19.50,18.25,37.05,41.75,AK_Bethel,43.70,...,34.45,42.95,19.45,7.30,5.1,58.65,10.95,61.9,AK | BETHEL CENSUS AREA,02050
4,11.75,79.90,68.10,60.75,33.10,30.00,32.10,59.25,AK_Bristol Bay,28.20,...,20.95,36.00,11.55,3.30,3.3,60.50,4.40,62.8,AK | BRISTOL BAY BOROUGH,02060


## 2 Peprocess the County urban vs rural dataset

In [12]:
rural_v_urban_county = pd.read_excel(r'/mnt/d/OneDrive - University College London/T1/CASA0007/0. Research Project/cleaned_data/2020_UA_COUNTY.xlsx', sheet_name = '2020_UA_COUNTY')

In [17]:
rural_v_urban_county.head()

,STATE,COUNTY,STATE_NAME,COUNTY_NAME,POP_COU,HOU_COU,ALAND_COU,ALAND_Mi²_COU,POPDEN_COU,HOUDEN_COU,...,POPPCT_RUR,HOU_RUR,HOUPCT_RUR,ALAND_RUR,ALAND_Mi²_RUR,ALAND_PCT_RUR,POPDEN_RUR,HOUDEN_RUR,RURALBLOCKS,GEOID
0,01,001,Alabama,Autauga,58805,24350,1539634184,594.452758,98.922916,40.962044,...,0.406768,9991,0.410308,1483727020,572.867002,0.963688,41.754892,17.440348,991,01001
1,01,003,Alabama,Baldwin,231767,124148,4117656199,1589.827058,145.781265,78.088997,...,0.375865,40740,0.328157,3762600021,1452.739868,0.913772,59.964624,28.043562,3181,01003
2,01,005,Alabama,Barbour,25223,11618,2292160149,885.003034,28.500467,13.127639,...,0.659200,7538,0.648821,2276027730,878.774307,0.992962,18.920672,8.577857,1011,01005
3,01,007,Alabama,Bibb,22293,9002,1612188717,622.466064,35.814001,14.461833,...,1.000000,9002,1.000000,1612188717,622.466064,1.000000,35.814001,14.461833,1090,01007
4,01,009,Alabama,Blount,59134,24622,1670259090,644.887035,91.696680,38.180330,...,0.904894,22337,0.907197,1658933117,640.514076,0.993219,83.542270,34.873551,2207,01009


In [14]:
rural_v_urban_county['STATE'] = rural_v_urban_county['STATE'].apply(lambda x: str(x).zfill(2))
rural_v_urban_county['COUNTY'] = rural_v_urban_county['COUNTY'].apply(lambda x: str(x).zfill(3))

In [16]:
rural_v_urban_county['GEOID'] = rural_v_urban_county['STATE'] + rural_v_urban_county['COUNTY']

|Field Name	| Field Description |
| --- | --- |
STATE | Two digit State Code
COUNTY | Three digit County Code
STATE_NAME | Name of the State
COUNTY_NAME | Name of the County or County equivalent.   (* in COUNTY_NAME indicates value(s) in this row were rounded up from 0.00 to 0.01 or down from 100 to 99.99)
POP_COU | 2020 Census total population of the County
HOU_COU	| 2020 Census total housing unit count of the County
ALAND_COU | 2020 land area of the County (square meters)
ALAND_Mi²_COU | 2020 land area of the County (square miles)
POPDEN_COU |	2020 population density of the County (square miles)
HOUDEN_COU |	2020 housing unit density of the County (square miles)
TOTALBLOCKS_COU |	2020 total blocks within the County
POP_URB |	2020 Census population of the County within Urban blocks
POPPCT_URB |	Percent of the 2020 Census population of the County within Urban blocks
HOU_URB |	2020 Census housing units in the County within Urban blocks
HOUPCT_URB |	Percent of 2020 Census housing units in the County within Urban blocks
ALAND_URB |	2020 Urban land area of the County (square meters)
ALAND_Mi²_URB |	2020 Urban land area of the County (square miles)
ALAND_PCT_URB |	Percent of 2020 land within the County that is classified as Urban
POPDEN_URB |	2020 Urban population density of the County (square miles)
HOUDEN_URB |	2020 Urban housing unit density of the County (square miles)
URBANBLOCKS |	2020 blocks classified as Urban within County
POP_RUR |	2020 Census population of the County within Rural blocks
POPPCT_RUR |	Percent of the 2020 Census population in the County within Rural blocks
HOU_RUR |	2020 Census housing units in the County within Rural blocks
HOUPCT_RUR |	Percent of the 2020 housing units in the County within Rural blocks
ALAND_RUR |	2020 Rural land area of the County (square meters)
ALAND_Mi²_RUR |	2020 Rural land area of the County (square miles)
ALAND_PCT_RUR |	Percent of 2020 land within the County that is classified as Rural
POPDEN_RUR |	2020 Rural population density of the County (square miles)
HOUDEN_RUR |	2020 Rural housing unit density of the County (square miles)
RURALBLOCKS |	2020 blocks classified as Rural within County


Selected data to join:

|Field Name	| Field Description |
| --- | --- |
| GEOID | Join on this column |
| POPPCT_URB |	Percent of the 2020 Census population of the County within Urban blocks |
| POPPCT_RUR |	Percent of the 2020 Census population in the County within Rural blocks |
| HOUPCT_URB |	Percent of 2020 Census housing units in the County within Urban blocks |
| HOUPCT_RUR |	Percent of the 2020 housing units in the County within Rural blocks |
|  |
|  |
|  |
|  |

In [43]:
county_rvu = rural_v_urban_county.filter(['GEOID', 'POPPCT_URB', 'POPPCT_RUR','HOUPCT_URB','HOUPCT_RUR'], axis=1)

In [44]:
county_rvu.head()

,GEOID,POPPCT_URB,POPPCT_RUR,HOUPCT_URB,HOUPCT_RUR
0,01001,0.593232,0.406768,0.589692,0.410308
1,01003,0.624135,0.375865,0.671843,0.328157
2,01005,0.340800,0.659200,0.351179,0.648821
3,01007,0.000000,1.000000,0.000000,1.000000
4,01009,0.095106,0.904894,0.092803,0.907197


## 3 Join two datasets

In [45]:
county_rvu.dtypes

GEOID          object
POPPCT_URB    float64
POPPCT_RUR    float64
HOUPCT_URB    float64
HOUPCT_RUR    float64
dtype: object

In [48]:
columns_to_multiply = ['POPPCT_URB','POPPCT_RUR','HOUPCT_URB','HOUPCT_RUR']

county_rvu[columns_to_multiply] = county_rvu[columns_to_multiply] * 100

In [51]:
county_rvu.head()

,GEOID,Urban_Pop_Percent,Rural_Pop_Percent,Urban_Housing_Percent,Rural_Housing_Percent
0,01001,59.323187,40.676813,58.969199,41.030801
1,01003,62.413545,37.586455,67.184328,32.815672
2,01005,34.080006,65.919994,35.117920,64.882080
3,01007,0.000000,100.000000,0.000000,100.000000
4,01009,9.510603,90.489397,9.280318,90.719682


In [50]:
county_rvu.columns = ['GEOID', 'Urban_Pop_Percent', 'Rural_Pop_Percent','Urban_Housing_Percent','Rural_Housing_Percent']

In [52]:
place_rvu_df = pd.merge(cdc_places, county_rvu, on='GEOID', how='inner')

In [53]:
place_rvu_df.shape

(3076, 44)

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
usa_map = gpd.read_file(r'/mnt/d/OneDrive - University College London/T1/CASA0007/0. Research Project/mapdata/cb_2018_us_county_500k.shp')

In [ ]:
# merge the map and our PLACE dataset
map_and_PLACE_data = usa_map.merge(cdc_20_21_plot, on="GEOID")

In [ ]:
title = 'Current asthma among adults aged >=18 years in the USA by County'
col = 'case_growth_rate'
source = 'Source: CDC_PLACE Data'


fig, ax = plt.subplots(1, figsize=(200, 80))
map_and_PLACE_data.plot(column="No leisure-time physical activity among adults aged >=18 years", cmap="viridis", linewidth=0.4, ax=ax, edgecolor=".5")

# Add a title
ax.set_title(title, fontdict={'fontsize': '25', 'fontweight': '3'})

# Create an annotation for the data source
ax.annotate(source, xy=(0.1, .08), xycoords='figure fraction', horizontalalignment='left', 
            verticalalignment='bottom', fontsize=10)
            
# Create colorbar as a legend
sm = plt.cm.ScalarMappable(norm=plt.Normalize(vmin=0, vmax=100), cmap="viridis")

# Empty array for the data range
sm._A = []

# Add the colorbar to the figure
cbaxes = fig.add_axes([0.07, 0.3, 0.01, 0.4])
cbar = fig.colorbar(sm, cax=cbaxes)


## 4 Save the combined data

In [ ]:
# Save to this r'/mnt/d/OneDrive - University College London/T1/CASA0007/0. Research Project/cleaned_data/combined/combined_cdc_r_v_u_df.csv'
place_rvu_df.to_csv(r'/mnt/d/OneDrive - University College London/T1/CASA0007/0. Research Project/cleaned_data/combined/combined_cdc_r_v_u_df.csv', index=False)